In [11]:
from pulp import *
from itertools import chain, combinations


# funkcija za podskupove
def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))


# funkcija za susjede
def neighbors(x):
    n = []
    for (i,j) in Eweak:
        if i==x and Eweak[i,j] == 1:
            n.append(j)
    return n
            
    

In [34]:
# primjer naseg grafa
V= [1, 2, 3, 4, 5, 6, 7, 8, 9]

# (u, v) grana cvorova u i v, ukoliko je 1 grana postoji, ukoliko je 0 ne
Eweak= { (1,2):1, (1,3):1, (1,4):0, (1,5):0, (1,6):0, (1,7):0, (1,8):0, (1,9):0,
         (2,1):1, (2,3):1, (2,4):1, (2,5):0, (2,6):0, (2,7):0, (2,8):0, (2,9):0,
         (3,1):1, (3,2):1, (3,4):1, (3,5):0, (3,6):0, (3,7):0, (3,8):0, (3,9):0,
         (4,1):0, (4,2):1, (4,3):1, (4,5):1, (4,6):1, (4,7):0, (4,8):0, (4,9):0,
         (5,1):0, (5,2):0, (5,3):0, (5,4):1, (5,6):1, (5,7):1, (5,8):1, (5,9):1,
         (6,1):0, (6,2):0, (6,3):0, (6,4):1, (6,5):1, (6,7):0, (6,8):0, (6,9):0,
         (7,1):0, (7,2):0, (7,3):0, (7,4):0, (7,5):1, (7,6):0, (7,8):0, (7,9):0,
         (8,1):0, (8,2):0, (8,3):0, (8,4):0, (8,5):1, (8,6):0, (8,7):0, (8,9):0,
         (9,1):0, (9,2):0, (9,3):0, (9,4):0, (9,5):1, (9,6):0, (9,7):0, (9,8):0}

In [52]:
# model - problem minimizacije
model = LpProblem("MinimumWeaklyConnectedDominatingSet", LpMinimize)

# varijable
x = LpVariable.dicts("x", Eweak, 0, 1, LpBinary)
y = LpVariable.dicts("y", V , 0, 1, LpBinary)

# funkcija cilja
model += 1 + lpSum( x [ (u, v) ]  for (u, v) in Eweak )

# ogranicenja 1, 2 i 3
for u in V:
    for v in V:
        if u!=v:
            model += x[ (u, v) ] <= Eweak[u,v]
            model += x[ (u, v) ] == x[ (v, u) ]
            model += y[u] + y[v] >= 2*x[ (u, v) ]

# ogranicenje 4
for S in powerset(V):
    for u in S:
        for v in S:
            if u!=v:
                model += lpSum( x[ (u, v) ] ) <= len(S)-1

# ogranicenje 5
for u in V:
    for v in V:
        if u!=v:
            model += lpSum( x[ (u,v) ]) == lpSum( y[t] for t in V) - 1

# ogranicenje 6
for v in V:
    for t in neighbors(v): 
        model += lpSum( y[t] ) >= 1

# rjesavanje
model.solve()

# ispis DS
selected = [u for u, var in y.items() if var.varValue == 1]
print("Selected:", selected)

#status:
print("Status solve: ", LpStatus[ model.status ])
print("Vars: \n")
for v in model.variables():
  print(v.name, "===> ", v.varValue)


print("Obj value: ", value(model.objective))


Selected: [2, 3, 4, 6]
Status solve:  Infeasible
Vars: 

x_(1,_2) ===>  3.0
x_(1,_3) ===>  3.0
x_(1,_4) ===>  3.0
x_(1,_5) ===>  3.0
x_(1,_6) ===>  3.0
x_(1,_7) ===>  3.0
x_(1,_8) ===>  3.0
x_(1,_9) ===>  3.0
x_(2,_1) ===>  3.0
x_(2,_3) ===>  3.0
x_(2,_4) ===>  3.0
x_(2,_5) ===>  3.0
x_(2,_6) ===>  3.0
x_(2,_7) ===>  3.0
x_(2,_8) ===>  3.0
x_(2,_9) ===>  3.0
x_(3,_1) ===>  3.0
x_(3,_2) ===>  3.0
x_(3,_4) ===>  3.0
x_(3,_5) ===>  3.0
x_(3,_6) ===>  3.0
x_(3,_7) ===>  3.0
x_(3,_8) ===>  3.0
x_(3,_9) ===>  3.0
x_(4,_1) ===>  3.0
x_(4,_2) ===>  3.0
x_(4,_3) ===>  3.0
x_(4,_5) ===>  3.0
x_(4,_6) ===>  3.0
x_(4,_7) ===>  3.0
x_(4,_8) ===>  3.0
x_(4,_9) ===>  3.0
x_(5,_1) ===>  3.0
x_(5,_2) ===>  3.0
x_(5,_3) ===>  3.0
x_(5,_4) ===>  3.0
x_(5,_6) ===>  3.0
x_(5,_7) ===>  3.0
x_(5,_8) ===>  3.0
x_(5,_9) ===>  3.0
x_(6,_1) ===>  3.0
x_(6,_2) ===>  3.0
x_(6,_3) ===>  3.0
x_(6,_4) ===>  3.0
x_(6,_5) ===>  3.0
x_(6,_7) ===>  3.0
x_(6,_8) ===>  3.0
x_(6,_9) ===>  3.0
x_(7,_1) ===>  3.0
x_(7,_2) ===